In [ ]:
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [ ]:
import os
print(os.path.exists("/content/drive/My Drive/Colab Notebooks/T5 copy of experiment.ipynb"))


True


In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import torch

import numpy as np

import csv


from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq
from transformers import pipeline

In [ ]:
NEWS_DATA_PATH = 'drive/My Drive/Colab Notebooks/期末/BBC News Summary(只有三個分類)/News Articles'
SUMMARIES_DATA_PATH = 'drive/My Drive/Colab Notebooks/期末/BBC News Summary(只有三個分類)/Summaries'

# 設定模型儲存路徑
MODEL_SAVE_PTH = 'drive/My Drive/Colab Notebooks/期末/二六.三個分類_5個epoch_T5'

# 設定設備為 GPU (如果可用的話) 或 CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定輸入和目標(摘要)的最大長度
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

# 設定批次大小
BATCH_SIZE = 8

# 定義情緒標籤
# SENTIMENT_LABELS = ["happy", "sad", "warn", "angry", "sorrow", "alert", "neutral"]
# 定義情緒標籤
SENTIMENT_LABELS = ["joy", "sadness", "anger", "fear", "surprise","neutral"]

# 建立情緒標籤到索引的對應字典
SENTIMENT_LABELS_MAP = {label: index for index, label in enumerate(SENTIMENT_LABELS)}



# 加入權重
summary_loss_weight = 0
sentiment_loss_weight = 1
prev_summary_loss = None
prev_sentiment_loss = None
avg_summary_loss = 0
avg_sentiment_loss = 0

**從root_path 加載數據**

根路徑下的每個子目錄(代表一個類別)，子目錄中的每個文件包含新聞內容。

返回一個包含所有新聞內容的 numpy 字符串數組。

News Articles：包含不同類別的新聞文章，每個類別是一個子資料夾，內有多個文件。

Summaries：包含每篇新聞文章的五個摘要。

pbar = tqdm(os.listdir(news_dir_path))：

這行程式碼將 news_dir_path 目錄中的所有文件名稱傳遞給 tqdm，以初始化進度條 pbar。在迴圈中使用 pbar 可以在迴圈每次迭代時更新進度條。



*   os.listdir(root_path) 返回根目錄下的所有子目錄名稱，假設每個子目錄代表一個類別。
*   os.path.join(root_path, cls) 獲取每個子目錄的完整路徑。


In [ ]:
def load_data(root_path):
    print('Loading: {}'.format(root_path))  # 打印正在載入的路徑

    data = []  # 初始化數據列表，用來存儲加載的數據
    classes = sorted(os.listdir(root_path))  # 確保類別按順序排序
    # 遍歷根目錄下的每個子目錄，假設每個子目錄代表一個類別。
    for cls in classes:  # 確保類別的順序一致
        news_dir_path = os.path.join(root_path, cls)  # 獲取類別資料夾的路徑

        # 使用自然排序的方法來確保文件按數字順序排序
        files = sorted(os.listdir(news_dir_path), key=lambda x: int(os.path.splitext(x)[0]))  # 確保文件按數字順序排序

        pbar = tqdm(files)   # 初始化進度條，用來顯示加載每個子目錄中文件的進度。

        # 驗證文件順序
        print(f'Class: {cls}')
        print('Files:', files)

        # 遍歷類別資料夾中的文件
        for news_file_name in pbar:
            news_file_path = os.path.join(news_dir_path, news_file_name)  # 獲取新聞文件的路徑

            # 打開新聞文件並讀取內容
            with open(news_file_path, 'r', encoding='unicode_escape') as file:
                lines = file.read().strip()
                if lines:
                    data.append(lines.replace('\n', ' '))  # 將內容中的換行符替換為空格，並添加到數據列表中。
                else:
                    print(f"Empty file found: {news_file_path}")

            # 更新進度條的後綴資訊
            if news_file_name == pbar.iterable[-1]: # 如果當前文件是目錄中的最後一個文件，更新進度條的後綴信息。
                pbar.set_postfix_str('{} class loaded'.format(cls)) # 設置進度條的後綴信息，顯示當前類別已加載完成。

    return np.array(data, dtype=str)  # 返回數據陣列



In [ ]:
# 載入新聞語料
news_corpus = load_data(NEWS_DATA_PATH)

# 載入摘要語料
summaries_corpus = load_data(SUMMARIES_DATA_PATH)

Loading: drive/My Drive/Colab Notebooks/期末/BBC News Summary(只有三個分類)/News Articles


  0%|          | 0/417 [00:00<?, ?it/s]

Class: politics
Files: ['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '070.txt', '071.txt', '072.txt', '073.txt', '074.txt', '075.txt', '076.txt', '077.txt', '078.txt', '079.txt', '080.txt', '081.txt', '082.txt', '083.txt', '084.txt', '085.txt', '086.txt', '087.txt', '088.txt', '089.txt

  0%|          | 0/511 [00:00<?, ?it/s]

Class: sport
Files: ['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '070.txt', '071.txt', '072.txt', '073.txt', '074.txt', '075.txt', '076.txt', '077.txt', '078.txt', '079.txt', '080.txt', '081.txt', '082.txt', '083.txt', '084.txt', '085.txt', '086.txt', '087.txt', '088.txt', '089.txt', 

...

  0%|          | 0/511 [00:00<?, ?it/s]

Class: sport
Files: ['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '070.txt', '071.txt', '072.txt', '073.txt', '074.txt', '075.txt', '076.txt', '077.txt', '078.txt', '079.txt', '080.txt', '081.txt', '082.txt', '083.txt', '084.txt', '085.txt', '086.txt', '087.txt', '088.txt', '089.txt', 

  0%|          | 0/401 [00:00<?, ?it/s]

Class: tech
Files: ['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt', '051.txt', '052.txt', '053.txt', '054.txt', '055.txt', '056.txt', '057.txt', '058.txt', '059.txt', '060.txt', '061.txt', '062.txt', '063.txt', '064.txt', '065.txt', '066.txt', '067.txt', '068.txt', '069.txt', '070.txt', '071.txt', '072.txt', '073.txt', '074.txt', '075.txt', '076.txt', '077.txt', '078.txt', '079.txt', '080.txt', '081.txt', '082.txt', '083.txt', '084.txt', '085.txt', '086.txt', '087.txt', '088.txt', '089.txt', '

100%|██████████| 401/401 [00:06<00:00, 61.03it/s, tech class loaded] 


In [ ]:
# 分割資料集為訓練集和測試集
# 5% 的資料用作測試集，剩下的 95% 用作訓練集
X_train, X_test, y_train, y_test = train_test_split(news_corpus, summaries_corpus, test_size=0.05, random_state=42)

# 將訓練集進一步分割為訓練集和驗證集
# 之前的訓練集再一次分割成新的訓練集和驗證集。
# test_size=0.05 表示將 5% 的訓練集數據用作驗證集，剩下的 95% 繼續用作訓練集。
X_train, X_vaild, y_train, y_vaild = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

In [ ]:
'''
# 假設 news_corpus 和 summaries_corpus 已經加載
# 創建一個包含原始數據集索引的數組
indices = np.arange(len(news_corpus))

# 第一次分割，用於獲取訓練集和測試集
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    news_corpus, summaries_corpus, indices, test_size=0.02, random_state=42)

# 第二次分割，用於獲取訓練集和驗證集
X_train, X_valid, y_train, y_valid, train_indices, valid_indices = train_test_split(
    X_train, y_train, train_indices, test_size=0.02, random_state=42)

# 現在可以打印出測試集和驗證集的索引
print("Test indices:", test_indices)
print("Validation indices:", valid_indices)
'''

'\n# 假設 news_corpus 和 summaries_corpus 已經加載\n# 創建一個包含原始數據集索引的數組\nindices = np.arange(len(news_corpus))\n\n# 第一次分割，用於獲取訓練集和測試集\nX_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(\n    news_corpus, summaries_corpus, indices, test_size=0.02, random_state=42)\n\n# 第二次分割，用於獲取訓練集和驗證集\nX_train, X_valid, y_train, y_valid, train_indices, valid_indices = train_test_split(\n    X_train, y_train, train_indices, test_size=0.02, random_state=42)\n\n# 現在可以打印出測試集和驗證集的索引\nprint("Test indices:", test_indices)\nprint("Validation indices:", valid_indices)\n'

In [ ]:
# 計算測試集、驗證集和訓練集的長度
len(X_test), len(X_vaild), len(X_train)

(180, 144, 572)

**自定義 PyTorch 的 Dataset 類，用於處理新聞和摘要數據。**

初始化數據集，包括輸入數據（新聞）和目標數據（摘要）。
定義數據集的大小和索引操作。

返回值：

    __len__ 返回數據集的大小。

    __getitem__ 根據索引返回對應的數據對（新聞和摘要）。

In [ ]:
# 定義資料集類別
class CorpusDataset(Dataset):
    def __init__(self, X_corpus, y_corpus):
        self.X_corpus = X_corpus  # 輸入數據
        self.y_corpus = y_corpus  # 目標數據

    def __len__(self):
        return len(self.X_corpus)  # 返回數據集的大小

    def __getitem__(self, index):
        return self.X_corpus[index], self.y_corpus[index]  # 根據索引返回對應的數據對


In [ ]:
# 創建訓練、驗證和測試資料集
train_model_inputs = CorpusDataset(X_train, y_train)
vaild_model_inputs = CorpusDataset(X_vaild, y_vaild)
test_model_inputs = CorpusDataset(X_test, y_test)

In [ ]:
# 創建訓練資料加載器
train_dataloader = DataLoader(
    train_model_inputs,
    # 避免局部最小
    shuffle=True,  # 訓練時打亂數據
    batch_size=BATCH_SIZE,  # 每個批次的大小
    drop_last=True,  # 如果最後一個批次數據不夠，則丟棄
    pin_memory=True  # 將數據複製到 CUDA 固定內存
)

# 创建验证数据加载器
eval_dataloader = DataLoader(
    vaild_model_inputs,
    shuffle=False,  # 驗證時不打亂數據
    # batch_size=BATCH_SIZE,  # 每個批次的大小
    drop_last=True,  # 如果最後一個批次數據不夠，則丟棄
    pin_memory=True  # 將數據複製到 CUDA 固定內存
)

# 创建测试数据加载器
test_dataloader = DataLoader(
    test_model_inputs,
    shuffle=False,
    batch_size=1,
    drop_last=True,
    pin_memory=True
)


In [ ]:
# 打印批次数量
print(f"Number of training batches: {len(train_dataloader)}")
print(f"Number of validation batches: {len(eval_dataloader)}")
print(f"Number of test batches: {len(test_dataloader)}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

summary_tokenizer = AutoTokenizer.from_pretrained("braindao/flan-t5-cnn", device=DEVICE)
summary_model = AutoModelForSeq2SeqLM.from_pretrained("braindao/flan-t5-cnn").to(DEVICE)

# 加載情感分析模型
sentiment_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# 設定訓練的週期數，每個週期包括對整個訓練數據集的一次完整遍歷。
num_train_epochs = 5

# 計算每個週期的更新步數
num_update_steps_per_epoch = len(train_dataloader)
# 總的訓練步數
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# 定義數據整理器
# **DataCollatorForSeq2Seq 是專門為序列到序列（Seq2Seq），包含標記化、填充
data_collator = DataCollatorForSeq2Seq(summary_tokenizer, model=summary_model)

# 定義優化器
# **summary_model.parameters()：將 summary_model 的所有參數傳遞給優化器
optimizer = AdamW(summary_model.parameters(), lr=1e-5)

# 損失函數
# 計算模型預測結果與實際標籤之間的差異
loss_fcn = CrossEntropyLoss()

**get_model_inputs 函數:將摘要文本轉換為模型的目標輸入格式**




*   tokenizer：用於標記化文本的 tokenizer，通常是預訓練模型的 tokenizer，例如 BART 或 BERT。
*   corpus：一個包含多個文本的列表，每個文本都需要進行標記化處理。
*   is_summaries：一個布爾值，指示是否處理的是摘要。如果是摘要，則使用不同的最大長度。


In [ ]:
# 定義資料處理函數
def get_model_inputs(tokenizer, corpus, is_summaries=False):
    model_inputs = []  # 初始化模型輸入列表

    # 設定最大長度，摘要用 MAX_TARGET_LENGTH，其他用 MAX_INPUT_LENGTH
    # MAX_TARGET_LENGTH與MAX_INPUT_LENGTH在最前面定義了
    max_length = MAX_INPUT_LENGTH if not is_summaries else MAX_TARGET_LENGTH

    # 對每個文本進行標記
    for text in corpus:
        model_inputs.append(tokenizer(
            text,
            max_length=max_length,
            truncation=True,  # 超過最大長度時進行截斷
        ))

    return model_inputs  # 返回標記化的輸入數據

# labels 是經過 get_model_inputs 函數和 data_collator 函數處理後的輸出
# 包含了轉換為標記 ID 的摘要文本（input_ids）和相應的注意力掩碼（attention_mask）。
# 這些張量將被用作目標數據，傳遞給模型用於計算損失。

'''
像是
{
    'input_ids': tensor([[101, 2023, 2003, 1037, 7099,  ...], [101, 2129, 2079, 2017, 2424, ...]]),
    'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, ...], [1, 1, 1, 1, 1, 1, 1, ...]])
}

'''

"\n像是\n{\n    'input_ids': tensor([[101, 2023, 2003, 1037, 7099,  ...], [101, 2129, 2079, 2017, 2424, ...]]),\n    'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, ...], [1, 1, 1, 1, 1, 1, 1, ...]])\n}\n\n"

In [ ]:
def get_sentiment_logits(sentiment_model, corpus):
    corpus_logits = []  # 存儲每個文本的情感 logits。

    with torch.no_grad():  # 不計算梯度(不需要進行反向傳播)，節省內存和計算資源
        for text in corpus:
            if text:  # 新增檢查空文本的邏輯
              sentiment = sentiment_model(text, return_all_scores=True)  # 獲取情感分析結果，返回所有分數。
              logits = [score['score'] for score in sentiment]  # 獲取每個情感標籤的分數作為 logits。
              corpus_logits.append(torch.tensor(logits))  # 將 logits 轉換為張量並添加到列表
            else:
                print(f"Empty text found in corpus")  # 新增警告信息
    return torch.stack(corpus_logits)  # 將所有 logits 張量堆疊成一個張量


'''
CrossEntropyLoss 的預期輸入（即 logits）
但 gen_summaries_sentiment_probs 經過 softmax 轉換為概率分數。
不符合CrossEntropyLoss 的預期輸入（即 logits）
'''
# 定義情感機率，返回每個情感標籤的概率分數。
def get_sentiment_probs(sentiment_model, corpus):
    corpus_scores = [] # 存儲每個文本的情感分數。

    with torch.no_grad():  # 不計算梯度(不需要進行反向傳播)，節省內存和計算資源
        for text in corpus:
            sentiment = sentiment_model(text, SENTIMENT_LABELS)  # 獲取情感分析結果，返回一個包含情感標籤和分數的字典。
            labels = sentiment['labels']
            scores = sentiment['scores']
            sorted_scores = [scores[labels.index(label)] for label in SENTIMENT_LABELS]  # 按情感標籤順序排列分數
            corpus_scores.append(torch.tensor(sorted_scores))  # 將分數轉換為張量並添加到列表

    return torch.stack(corpus_scores)  # 將所有分數張量堆疊成一個張量

# 將機率轉為情感標籤
def get_sentiment_labels(sentiment_model, corpus):
    corpus_scores = []

    with torch.no_grad():  # 禁用梯度計算，節省內存和計算資源
        for text in corpus:
            if text:  # 新增檢查空文本的邏輯
              sentiment = sentiment_model(text, SENTIMENT_LABELS)  # 使用情感模型進行情感分析，輸出結果包含各情感標籤的分數。
              label = sentiment['labels'][0]  # 獲取最高機率的情感標籤
              corpus_scores.append(SENTIMENT_LABELS_MAP[label])  # 將獲取到的情感標籤轉換為數字並添加到列表
            # SENTIMENT_LABELS_MAP 是一個字典，將情感標籤轉換為數字。
            else:
                print(f"Empty text found in corpus")  # 新增警告信息
    return torch.tensor(corpus_scores)  # 將所有標籤轉換為張量

**七 居批踢建議**

**十四**情感損失大增加權重

In [ ]:
def adjust_weights(avg_train_summary_loss, avg_train_sentiment_loss, avg_valid_summary_loss, avg_valid_sentiment_loss, min_step=0.2, max_step=0.4):
    # 結合訓練和驗證損失，給予兩者一定的權重（如各佔50%）
    combined_summary_loss = 0.5 * avg_train_summary_loss + 0.5 * avg_valid_summary_loss
    combined_sentiment_loss = 0.5 * avg_train_sentiment_loss + 0.5 * avg_valid_sentiment_loss

    # 計算損失比例差異
    if combined_summary_loss + combined_sentiment_loss > 0:
        loss_difference = abs(combined_summary_loss - combined_sentiment_loss) / (combined_summary_loss + combined_sentiment_loss)
    else:
        loss_difference = 0

    # 增加步長計算中的影響係數，使調整幅度更大
    step = min(min_step * (1 + 5 * loss_difference**2), max_step)

    # 動態調整權重
    if combined_summary_loss > combined_sentiment_loss:
        # 當摘要損失大於情感損失，應增加摘要損失的權重
        summary_weight = min(0.8, 0.5 + step)
        sentiment_weight = max(0.2, 0.5 - step)
    else:
        # 當情感損失大於摘要損失，應增加情感損失的權重
        summary_weight = max(0.2, 0.5 - step)
        sentiment_weight = min(0.8, 0.5 + step)

    return summary_weight, sentiment_weight


**損失標準化**

In [ ]:
def standardize_loss(loss, mean_loss, offset=0.01):
    if mean_loss == 0:
        return torch.tensor(loss + offset, dtype=torch.float32).to(DEVICE)
    standardized_loss = (loss - mean_loss) / (abs(mean_loss) + 1e-9)
    standardized_loss = max(standardized_loss + offset, 0.0)  # 确保标准化后的损失值为非负
    return torch.tensor(standardized_loss, dtype=torch.float32).to(DEVICE)


**10.**

**十一**

**十二**

In [ ]:
def test_step(epoch, use_generated_summaries=True):
    summary_model.eval()  # 設置模型為評估模式

    same_sentiment_count = 0  # 初始化相同情感計數
    total_count = 0  # 初始化總計數
    result = []  # 初始化存儲情感一致性的結果

    pbar = tqdm(test_dataloader)  # 初始化進度條
    pbar.set_description_str('[Epoch {}] Test'.format(epoch))

    csv_file = f'/content/drive/My Drive/Colab Notebooks/期末/二三/test_generated_summaries_epoch_{epoch}.csv'
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Original News', 'Original Summary', 'Generated Summary', 'News Sentiment', 'Generated Summary Sentiment', 'Same Sentiment'])

        for news, summaries in pbar:  # 遍歷測試數據加載器中的每個批次數據。news 是新聞文本，summaries 是資料集中的摘要。
            if use_generated_summaries:
                # 獲取輸入數據的ID並移動到設備
                inputs = summary_tokenizer(news, return_tensors='pt', padding=True, truncation=True, max_length=512)
                summary_model_inputs = inputs['input_ids'].to(DEVICE)
                attention_mask = inputs['attention_mask'].to(DEVICE)

                # 生成摘要
                summary_model_outputs = summary_model.generate(summary_model_inputs, attention_mask=attention_mask, max_length=512)
                # 解碼生成的摘要文本
                summarized_text = summary_tokenizer.batch_decode(summary_model_outputs, skip_special_tokens=True)
            else:
                # 使用資料集中的原始摘要
                summarized_text = summaries

            # 獲取新聞的情感標籤
            news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
            summarized_sentiment_labels = get_sentiment_labels(sentiment_model, summarized_text).to(DEVICE)

            # 計算情感損失
            gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, summarized_text).to(DEVICE)
            gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs.clamp(min=1e-9)).to(DEVICE)

            # 比較新聞和摘要的情感標籤是否一致
            same_sentiment = (torch.argmax(gen_summaries_sentiment_logits, dim=1) == news_sentiment_labels).sum().item()
            same_sentiment_count += same_sentiment
            total_count += len(news_sentiment_labels)

            writer.writerow([
                epoch,
                news[0],
                summaries[0],
                summarized_text[0] if use_generated_summaries else summaries[0],
                news_sentiment_labels[0].item(),
                torch.argmax(gen_summaries_sentiment_logits, dim=1)[0].item(),
                same_sentiment
            ])

            # 更新進度條後綴，顯示相同情感標籤的比例
            same_sentiment_rate = same_sentiment_count / total_count if total_count > 0 else 0
            pbar.set_postfix_str('Same Sentiment Rate: {:.3f}'.format(same_sentiment_rate))

    print(f"Final Same Sentiment Rate: {same_sentiment_rate:.3f}")



**3. 標準化損失和動態調整權重**

**10.**

**十一**

**十二**

In [ ]:
import csv

def train_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight):
    global prev_summary_loss, prev_sentiment_loss
    total_training_loss = []
    total_summary_loss = []
    total_sentiment_loss = []
    same_sentiment_count = 0
    total_count = 0

    summary_model.train()
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    # 確保每個 epoch 都創建新的 CSV 文件並在第一行寫入標題
    csv_file = f'/content/drive/My Drive/Colab Notebooks/期末/二三/train_generated_summaries_epoch_{epoch}.csv'
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Original News', 'Original Summary', 'Generated Summary', 'News Sentiment', 'Generated Summary Sentiment', 'Same Sentiment'])

        for news, summaries in pbar:
            if news is None or summaries is None:
                print("Error: News or Summaries is None")
                continue
            inputs = data_collator(get_model_inputs(summary_tokenizer, news))
            X_input_ids = inputs['input_ids'].to(DEVICE)
            X_attention_mask = inputs['attention_mask'].to(DEVICE)
            labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
            y_input_ids = labels['input_ids'].to(DEVICE)

            summary_loss = summary_model(input_ids=X_input_ids, attention_mask=X_attention_mask, labels=y_input_ids).loss

            gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids, max_new_tokens=MAX_TARGET_LENGTH), skip_special_tokens=True)

            if gen_summaries is None:
                print("Error: Generated summaries is None")
                continue

            news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
            gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
            sentiment_loss = loss_fcn(gen_summaries_sentiment_probs, news_sentiment_labels)

            # 標準化損失
            standardized_summary_loss = standardize_loss(summary_loss.item(), avg_summary_loss)
            standardized_sentiment_loss = standardize_loss(sentiment_loss.item(), avg_sentiment_loss)

            total_loss = summary_loss_weight * summary_loss + sentiment_loss_weight * sentiment_loss

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            total_training_loss.append(total_loss.item())
            total_summary_loss.append(summary_loss.item())
            total_sentiment_loss.append(sentiment_loss.item())

            same_sentiment = (torch.argmax(gen_summaries_sentiment_probs, dim=1) == news_sentiment_labels).sum().item()
            same_sentiment_count += same_sentiment
            total_count += len(news_sentiment_labels)

            for i in range(len(news)):
                writer.writerow([
                    epoch,
                    news[i],
                    summaries[i],
                    gen_summaries[i],
                    news_sentiment_labels[i].item(),
                    torch.argmax(gen_summaries_sentiment_probs, dim=1)[i].item(),
                    1 if news_sentiment_labels[i] == torch.argmax(gen_summaries_sentiment_probs, dim=1)[i] else 0
                ])
            pbar.set_postfix_str('Batch Loss: {:.6f}, Avg Loss: {:.6f}'.format(total_loss.item(), sum(total_training_loss) / len(total_training_loss)))



    avg_train_loss = sum(total_training_loss) / len(total_training_loss)
    avg_summary_loss = 0.9 * avg_summary_loss + 0.1 * (sum(total_summary_loss) / len(total_summary_loss))
    avg_sentiment_loss = 0.9 * avg_sentiment_loss + 0.1 * (sum(total_sentiment_loss) / len(total_sentiment_loss))
    print(f"Training Average Loss: {avg_train_loss:.6f}")
    print(f"Average Summary Loss: {avg_summary_loss:.6f}")
    print(f"Average Sentiment Loss: {avg_sentiment_loss:.6f}")

    same_sentiment_rate = same_sentiment_count / total_count if total_count > 0 else 0
    print(f"Same Sentiment Rate: {same_sentiment_rate:.3f}")
    return avg_train_loss,avg_summary_loss, avg_sentiment_loss



**4. 滑動平均能夠平滑損失值，減少波動，使得損失值變得更加穩定和可追踪。**

**5.**

**10.**

**十一**

**十二**

In [ ]:
def eval_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight):
    total_valid_loss = []
    total_summary_loss = []
    total_sentiment_loss = []
    same_sentiment_count = 0
    total_count = 0

    summary_model.eval()

    pbar = tqdm(eval_dataloader)
    pbar.set_description_str('[Epoch {}] Validation'.format(epoch))

    csv_file = f'/content/drive/My Drive/Colab Notebooks/期末/二三/valid_generated_summaries_epoch_{epoch}.csv'
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Original News', 'Original Summary', 'Generated Summary', 'News Sentiment', 'Generated Summary Sentiment', 'Same Sentiment'])

        for news, summaries in pbar:
            inputs = data_collator(get_model_inputs(summary_tokenizer, news))
            X_input_ids = inputs['input_ids'].to(DEVICE)
            X_attention_mask = inputs['attention_mask'].to(DEVICE)
            labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
            y_input_ids = labels['input_ids'].to(DEVICE)

            summary_loss = summary_model(input_ids=X_input_ids, attention_mask=X_attention_mask, labels=y_input_ids).loss
            gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids, max_new_tokens=MAX_TARGET_LENGTH), skip_special_tokens=True)

            news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
            gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
            gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs.clamp(min=1e-9))

            sentiment_loss = loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

            standardized_summary_loss = standardize_loss(summary_loss.item(), avg_summary_loss)
            standardized_sentiment_loss = standardize_loss(sentiment_loss.item(), avg_sentiment_loss)

            total_loss = summary_loss_weight * standardized_summary_loss + sentiment_loss_weight * standardized_sentiment_loss

            total_valid_loss.append(total_loss.item())
            total_summary_loss.append(summary_loss.item())
            total_sentiment_loss.append(sentiment_loss.item())

            for news_sentiment, summary_sentiment in zip(news_sentiment_labels, torch.argmax(gen_summaries_sentiment_logits, dim=1)):
                same_sentiment = 1 if news_sentiment == summary_sentiment else 0
                same_sentiment_count += same_sentiment
                total_count += 1
                writer.writerow([
                    epoch,
                    news[0],
                    summaries[0],
                    gen_summaries[0],
                    news_sentiment.item(),
                    summary_sentiment.item(),
                    same_sentiment
                ])

            pbar.set_postfix_str('Batch Loss: {:.6f}, Avg Loss: {:.6f}'.format(total_loss.item(), sum(total_valid_loss) / len(total_valid_loss)))

    avg_valid_loss = sum(total_valid_loss) / len(total_valid_loss)
    avg_summary_loss = 0.9 * float(avg_summary_loss) + 0.1 * (sum(total_summary_loss) / len(total_summary_loss))
    avg_sentiment_loss = 0.9 * float(avg_sentiment_loss) + 0.1 * (sum(total_sentiment_loss) / len(total_sentiment_loss))
    print(f"Validation Average Loss: {avg_valid_loss:.6f}")
    print(f"Average Summary Loss: {avg_summary_loss:.6f}")
    print(f"Average Sentiment Loss: {avg_sentiment_loss:.6f}")

    same_sentiment_rate = same_sentiment_count / total_count if total_count > 0 else 0
    print(f"Same Sentiment Rate: {same_sentiment_rate:.3f}")
    return avg_valid_loss, avg_summary_loss, avg_sentiment_loss


**6. 打印生成的摘要和檢查 Same Sentiment Rate**

**八**

**10.**

**十一**

**十二**

In [ ]:
# 定義一個函數用來從檔案名中解析出模型訓練的 epoch 數字
def get_model_epoch_num(file_name):
    # 從檔案名分割出 epoch 數字，並轉換成整數型態返回
    '''
    model_epoch_10.pt
    file_name.split('_') ["model", "epoch", "10.pt"]
    file_name.split('_')[-1]  "10.pt"
    file_name.split('_')[-1][:-3]  "10"
    '''
    return int(file_name.split('_')[-1][:-3])

# 用來存儲最新的模型檢查點的字典
ckpt = None

# 檢查模型儲存路徑是否存在，不存在則創建
if not os.path.exists(MODEL_SAVE_PTH):
    os.mkdir(MODEL_SAVE_PTH)

# 檢查指定路徑下是否有檔案存在
if len(os.listdir(MODEL_SAVE_PTH)) != 0:
    latest = os.listdir(MODEL_SAVE_PTH)[0]  # 先預設第一個檔案為最新

    # 遍歷該路徑下的所有檔案
    for file in os.listdir(MODEL_SAVE_PTH):
        # 檢查檔案是否為 PyTorch 的模型檔案（以 'pt' 結尾），且檢查是否比已知的最新檔案新
        # file[-2:]:倒數第二個字符開始一直到字符串的結尾
        if file[-2:] == 'pt' and get_model_epoch_num(file) > get_model_epoch_num(latest):
            latest = file  # 更新最新檔案

    # 加載最新的模型檢查點
    # 模型保存路徑 MODEL_SAVE_PTH 和最新的檔案名 latest 組合成一個完整的文件路徑。
    '''
    latest 是最新的檢查點"檔案名稱"
    MODEL_SAVE_PTH 是保存"檔案的目錄"
    torch.load 函數將檢查點檔案加載到 ckpt 變數中，這是一個包含模型狀態和訓練進度的字典。
    '''
    ckpt = torch.load(os.path.join(MODEL_SAVE_PTH, latest))

start_epoch = 0  # 初始化訓練開始的 epoch 數

# 如果 ckpt 不是 None，即成功加載了模型檢查點
if ckpt != None:
    # 恢復訓練進度，可以從上次訓練結束的地方繼續訓練
    start_epoch = ckpt['epoch']  # 從檢查點讀取目前的 epoch 數
    # load_state_dict 將檢查點中的模型參數(權重、biases)，加載到 summary_model 模型中
    summary_model.load_state_dict(ckpt['model_state_dict'])


**全部印出來**

**十四**

In [ ]:
# 初始測試步驟，使用資料集中的原始摘要
# test_step(0)
# Final Same Sentiment Rate: 0.583


# 開始訓練和測試周期
for epoch in range(start_epoch + 1, start_epoch + num_train_epochs + 1):
    print(summary_loss_weight, sentiment_loss_weight)

    avg_training_loss, avg_train_summary_loss, avg_train_sentiment_loss = train_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight)
    print("\n")

    avg_valid_loss, avg_valid_summary_loss, avg_valid_sentiment_loss = eval_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight)
    print("\n")

    summary_loss_weight, sentiment_loss_weight = adjust_weights(
        avg_train_summary_loss,
        avg_train_sentiment_loss,
        avg_valid_summary_loss,
        avg_valid_sentiment_loss,
        min_step=0.1,
        max_step=0.2
    )
    print(f"Adjusted weights - Summary: {summary_loss_weight}, Sentiment: {sentiment_loss_weight}")

    test_step(epoch)

    torch.save({
        'epoch': epoch,
        'model_state_dict': summary_model.state_dict()
    }, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))


0 1


[Epoch 3] Training: 100%|██████████| 149/149 [19:06<00:00,  7.70s/it, Batch Loss: 1.535116, Avg Loss: 1.577509]


Training Average Loss: 1.577509
Average Summary Loss: 0.078751
Average Sentiment Loss: 0.157751
Same Sentiment Rate: 0.648




[Epoch 3] Validation: 100%|██████████| 64/64 [02:57<00:00,  2.77s/it, Batch Loss: 0.574185, Avg Loss: 1.301089]


Validation Average Loss: 1.301089
Average Summary Loss: 0.084226
Average Sentiment Loss: 0.129109
Same Sentiment Rate: 0.578


Adjusted weights - Summary: 0.36207859792155295, Sentiment: 0.637921402078447


...

[Epoch 5] Validation: 100%|██████████| 64/64 [03:11<00:00,  3.00s/it, Batch Loss: 0.727112, Avg Loss: 1.108745]


Validation Average Loss: 1.108745
Average Summary Loss: 0.078608
Average Sentiment Loss: 0.126958
Same Sentiment Rate: 0.609


Adjusted weights - Summary: 0.345205703177467, Sentiment: 0.654794296822533


[Epoch 5] Test: 100%|██████████| 67/67 [03:24<00:00,  3.05s/it, Same Sentiment Rate: 0.567]


Final Same Sentiment Rate: 0.567
0.345205703177467 0.654794296822533


[Epoch 6] Training:  47%|████▋     | 70/149 [09:00<10:16,  7.80s/it, Batch Loss: 1.227193, Avg Loss: 1.245930]

In [ ]:
import os
import torch

# 保存模型
def save_model(epoch, model, optimizer, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

# 加載模型
def load_model(path, model, optimizer):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    return model, optimizer, epoch


In [ ]:
test_step(0)

for epoch in range(start_epoch + 1, start_epoch + num_train_epochs + 1):
    # 動態調整權重
    summary_loss_weight, sentiment_loss_weight = adjust_weights(epoch, avg_summary_loss, avg_sentiment_loss, total_epochs=num_train_epochs)

    # 执行一个训练周期
    avg_summary_loss, avg_sentiment_loss = train_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight)

    # 保存模型狀態
    save_model(epoch, summary_model, optimizer, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))

    # 重新加載模型狀態以進行驗證
    summary_model, optimizer, _ = load_model(os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'), summary_model, optimizer)

    # 执行一个验证周期
    avg_valid_loss, avg_summary_loss, avg_sentiment_loss = eval_step(epoch, avg_summary_loss, avg_sentiment_loss, summary_loss_weight, sentiment_loss_weight)

    # 重新加載模型狀態以進行測試
    summary_model, optimizer, _ = load_model(os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'), summary_model, optimizer)

    # 執行一個測試周期
    test_step(epoch)


100%|██████████| 41/41 [00:53<00:00,  1.31s/it]


Final Same Sentiment Rate: 0.609756


[Epoch 1] Training: 100%|██████████| 92/92 [09:13<00:00,  6.02s/it, Batch Loss: 1.161780, Avg Loss: 1.231201]


Total number of batches processed in epoch 1: 92
Same Sentiment Rate: 0.533


[Epoch 1] Validation: 100%|██████████| 39/39 [00:50<00:00,  1.30s/it, Batch Loss: 0.313574, Avg Loss: 0.148322]


...

Final Same Sentiment Rate: 0.609756


[Epoch 3] Training: 100%|██████████| 92/92 [09:14<00:00,  6.03s/it, Batch Loss: 1.044674, Avg Loss: 1.111023]


Total number of batches processed in epoch 3: 92
Same Sentiment Rate: 0.546


[Epoch 3] Validation:  59%|█████▉    | 23/39 [00:29<00:20,  1.29s/it, Batch Loss: 0.424985, Avg Loss: 0.151832]


KeyboardInterrupt: 

**全部印出來**

In [ ]:
# 最後一區有用到 __測試集

# 定義一個函數來在測試集上評估模型
def test_step():
    # 將模型設置為評估模式
    summary_model.eval()

    # 初始化一個列表來存儲測試結果
    result = []

    # 初始化一個列表來存儲測試結果
    result_new = []

    # 初始化進度條
    pbar = tqdm(test_dataloader)

    # 遍歷測試數據集中的每個批次
    for index, (news, _) in enumerate(pbar):
        # 使用 data_collator 函數和分詞器處理新聞文本，生成模型輸入
        summary_model_inputs = data_collator(get_model_inputs(summary_tokenizer, news))['input_ids'].to(DEVICE)
        # 使用模型生成摘要
        summary_model_outputs = summary_model.generate(summary_model_inputs)
        # 將生成的摘要轉換為文本
        summarized_text = summary_tokenizer.batch_decode(summary_model_outputs, skip_special_tokens=True)

        # 打印每個生成的摘要
        print(f'news Text {index + 1}: {news}')

        # 打印每個生成的摘要
        print(f'Summarized Text {index + 1}: {summarized_text}')

                # 將結果添加到結果列表中
        result_new.append({
            'News': news[0],
            'Summarized Text': summarized_text,

        })

        # 使用情緒分析模型獲取新聞文本的情緒標籤
        news_sentiment_label = get_sentiment_labels(sentiment_model, news)[0]
        # 使用情緒分析模型獲取生成摘要的情緒標籤
        summarized_sentiment_label = get_sentiment_labels(sentiment_model, summarized_text)[0]

        # 比較新聞文本和生成摘要的情緒標籤是否相同，並將結果添加到結果列表中
        # result是一個列表來存儲測試結果 如果摘要和原文標籤相同就append1，最後計算1總共的數量
        result.append(1 if news_sentiment_label == summarized_sentiment_label else 0)

        # 如果當前批次是最後一個，更新進度條的後綴信息，顯示相同情緒的比例
        # 所以是1越多越好
        # Same Sentiment Rate: 摘要和原文標籤相同/總共原文數量
        if index == len(test_dataloader) - 1:
            pbar.set_postfix_str('Same Sentiment Rate: {:.3f}'.format(result.count(1) / len(result)))
        # 創建 pandas DataFrame 並顯示表格
    results_df = pd.DataFrame(result_new)
    print(results_df)

    # 將結果匯出成 CSV 文件
    results_df.to_csv('results.csv', index=False)

In [ ]:
test_step()

100%|██████████| 67/67 [01:38<00:00,  1.47s/it, Same Sentiment Rate: 0.328]
